In [2]:
import os
os.chdir("../")

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    epochs: int
    warmup_steps: int
    train_batch_size_per_device: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    evaluation_steps: int
    save_steps: int
    gradient_accumulation_steps: int


In [4]:
from textsummarizer.constants import *
from textsummarizer.utils.common_func import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_file_path = CONFIG_FILE_PATH, params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainingConfig:
        config = self.config.model_training
        params = self.params.TrainingArgs

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainingConfig(
            root_dir=Path(config.root_dir),
            data_path= config.data_path,
            model_ckpt= config.model_ckpt,
            epochs=params.epochs,
            warmup_steps= params.warmup_steps,
            train_batch_size_per_device= params.train_batch_size_per_device,
            weight_decay=params.weight_decay,
            logging_steps= params.logging_steps,
            evaluation_strategy= params.evaluation_strategy,
            evaluation_steps = params.evaluation_steps,
            save_steps= params.save_steps,
            gradient_accumulation_steps= params.gradient_accumulation_steps

        )

        return model_trainer_config

In [5]:
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

class ModelTrainer:
    def __init__(self, config: ModelTrainingConfig):
        self.config = config

    def train_model(self):
        my_device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"Using device: {my_device}")
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        pegasus_model = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(my_device)
        seg2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=pegasus_model)

        dataset = load_from_disk(self.config.data_path)

        training_arguments = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=self.config.epochs,
            per_device_train_batch_size=self.config.train_batch_size_per_device,
            warmup_steps=self.config.warmup_steps,
            weight_decay=self.config.weight_decay,
            logging_steps=self.config.logging_steps,
            do_eval=True,
            save_steps=self.config.save_steps,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps,
        )

        trainer = Trainer(
            model=pegasus_model.to(my_device),
            args=training_arguments,
            data_collator=seg2seq_data_collator,
            train_dataset=dataset["test"],
            eval_dataset=dataset["validation"],
        )

        trainer.train()

        pegasus_model.save_model(os.path.join(self.config.root_dir, "pegasus_model_trained"))

        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "pegasus_model_tokenizer"))


        


/opt/anaconda3/envs/textSummarizer2/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-05-18 16:00:34,949: INFO:  config:  PyTorch version 2.7.0 available.]


In [12]:

PYTORCH_MPS_HIGH_WATERMARK_RATIO = 0.0

In [13]:
try:
    config = ConfigurationManager()
    config_data_transformation = config.get_model_trainer_config()
    data_transformation = ModelTrainer(config = config_data_transformation)
    data_transformation.train_model()
except Exception as e:
    raise e

[2025-05-18 16:21:23,760: INFO:  common_func:  Yaml file: config/config.yaml has been loaded succesfully]
[2025-05-18 16:21:23,823: INFO:  common_func:  Yaml file: params.yaml has been loaded succesfully]
[2025-05-18 16:21:23,854: INFO:  common_func:  created directory at: artifacts]
[2025-05-18 16:21:23,878: INFO:  common_func:  created directory at: artifacts/model_training]
Using device: cpu


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: MPS backend out of memory (MPS allocated: 6.08 GB, other allocations: 2.94 GB, max allowed: 9.07 GB). Tried to allocate 375.40 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

zsh:1: unmatched '
